In [1]:
import os
from transformers import TrainingArguments
from trl import DPOTrainer

from datasets import load_dataset,Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from vllm import SamplingParams, LLM
#from src.utils import gen_prompt,get_specific_batch
#from src.dpo import run_dpo
import json

generation=0
batch_size=64
model_id = "team-hatakeyama-phase2/8B-nishijima-tanuki8b_dpo_full_001-checkpoint-137"
sft_dataset_name="kanhatakeyama/0717-calm3-22b-random-genre-inst-sft-tsub-part"
in_jsonl_path=f"out/{generation}.jsonl"
max_model_len=2048

/home/hatakeyama/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2024-07-29 16:18:15,151] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


2024-07-29 16:18:15,939	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
sft_dataset_name="team-hatakeyama-phase2/synth-persona-jp-math-nemotron-4"
ds=load_dataset(sft_dataset_name,split="train")

Generating train split: 100%|██████████| 11392/11392 [00:00<00:00, 66003.40 examples/s]


In [7]:
ds[4]

{'messages': [{'role': 'user',
   'content': 'ゴミ処理プラント操作員の太郎さんは、毎日、プラントで処理するゴミの量を記録しています。ある週の記録によると、月曜日は8トンのゴミを処理し、火曜日は月曜日の1.5倍の量のゴミを処理しました。水曜日は火曜日の半分の量のゴミを処理し、木曜日は水曜日の2トン少ない量のゴミを処理しました。金曜日は木曜日の1.2倍の量のゴミを処理しました。\n\n1. この週にプラントで処理したゴミの合計量はいくつですか？\n2. この週にプラントで処理したゴミの1日あたりの平均量はいくつですか？'},
  {'role': 'assistant',
   'content': '1. この週にプラントで処理したゴミの合計量は、各日のゴミの量を合計することで求めることができます。\n\n   - 月曜日: 8トン\n   - 火曜日: 8 * 1.5 = 12トン\n   - 水曜日: 12 / 2 = 6トン\n   - 木曜日: 6 - 2 = 4トン\n   - 金曜日: 4 * 1.2 = 4.8トン (小数点以下は切り上げます) = 5トン\n\n   合計量は、8 + 12 + 6 + 4 + 5 = 35トン となります。\n\n2. この週にプラントで処理したゴミの1日あたりの平均量は、合計量を日数で割ることで求めることができます。\n\n   1日あたりの平均量 = 合計量 / 日数 = 35トン / 5日 ≈ 7トン/日\n\n   ただし、実際には毎日同じ量のゴミを処理できるわけではないので、この平均量はあくまで目安となります。'}]}

In [1]:
import wandb
mode = 'offline'
wandb.init(project="wandb_test",
           anonymous="allow",
		   mode=mode)



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


In [4]:

with open(in_jsonl_path,"r") as f:
    new_ds=[json.loads(line) for line in f]

dpo_dataset=Dataset.from_list(new_ds)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_id,device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id)


Loading checkpoint shards: 100%|██████████| 4/4 [00:09<00:00,  2.30s/it]


In [3]:

from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "team-hatakeyama-phase2/8B-nishijima-tanuki8b_dpo_full_001-checkpoint-137"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [7]:
txt="user: 第二次世界大戦中、日本の演劇界はどのような影響を受けましたか？また、その影響が現代の日本の演劇作品にどのように反映されているかについて考察してください。さらに、以下の条件を満たす台本のシーンを作成してください： 1. 登場人物AとBが登場し、Aは整数を、Bは小数を基にした議論をしている。 2. Aが整数が優れていると主張する一方で、Bは小数のほうが柔軟で便利だと反論している。 3. この会話を通じて観客に対して数学的な概念や歴史的背景を自然に学べるようにすること。 4. 最終的に両者が共通の理解に達するか、対立したまま終わるかの結末を選ぶこと。 5. 各キャラクターのセリフには具体的な例を交えて説得力を持たせること。 6. 全体のトーンは一貫性がありながらもユーモラスであること。 7. 指定された時間内に収まるよう工夫すること（例えば、約5分間のシーン）。 この制約の中で、あなたの創造力を最大限に活かしてください。 assistant: --- ### 日本演劇界の第二次世界大戦の影響と現代への反映 #### 戦時中の日本演劇界の状況 第二次世界大戦中、日本は厳しい統制下にありましたが、それでも演劇活動は続けられていました。しかし、戦争の影響で劇場設備の破壊や資材の不足が生じました。劇団も国家のプロパガンダに利用されることが多くなり、自由な創作活動が制限されました。そのため、多くの劇作家や俳優たちは困難な状況に直面しました。 #### 戦後の復興と再構築 戦後の日本では、GHQによる占領政策の下で文化復興が進みました。特に演劇においては、新しいスタイルやテーマが取り入れられ、社会の変革を反映した作品が多く生まれました。この時期、リアリズム演劇が主流となり、社会問題を扱った作品が注目を浴びました。これにより、演劇は再び市民生活の一部として重要な役割を果たすようになりました。 #### 現代への影響 現代の日本の演劇作品は、過去の戦争体験や社会問題をテーマに取り入れることが多いです。これは、過去の記憶や文化遺産を次世代に引き継ぐための手段でもあります。また、テクノロジーの進化に伴い、舞台演出にも新たな手法が導入され、より多様な表現が可能となっています。このように、戦時中の困難な経験が現在の創作活動に大きな影響を及ぼしています。 --- ### 台本シーンの作成 タイトル: 「数の世界の戦い」 登場人物: - A (アキラ): 中年の数学者、整数に情熱を持つ。 - B (ベティ): 年若いエンジニア、小数の柔軟性を評価する。 場所: アキラの研究室 --- [部屋の中央にあるテーブルを挟んで、アキラとベティが座っている。] アキラ: （ノートを広げながら）君たちの若い世代は、何でもかんでも「フレキシブル」とか言うけど、本当に重要なのは基礎だよ！整数はその基礎だ。 ベティ: でも先生、（タブレットを見せながら）、小数の方がもっと現実的ですよ。例えば、コンピュータープログラミングでは0.1を正確に表すのが難しいんです。 アキラ: それは確かに一理ある。だがね、整数は人類の歴史と共に歩んできたんだぞ。ピタゴラスの定理から何まで、全てが整数に基づいているんだ。 ベティ: だからって、全てが二進法で表されるべきなんてナンセンスじゃないですか。（笑いながら）それに、科学の進歩だって小数なしじゃ考えられませんよ。GPSの精度なんかまさにそうですよね。 アキラ: GPSの例を出すとは…なるほど、それは認めるしかないかな。ただ、それも結局は整数で計算されたアルゴリズムのおかげなんだよ。 ベティ: そんな風に言われると、なんだか納得しちゃうんですけど…。じゃあ、どうやってこの二つをうまく共存させればいいんでしょう？ アキラ: おお、いい質問だね。実はそれが数学の美しさなんだ。整数と浮動小数点の両方を使いこなすことで、私たちは完璧なバランスを見つけられるんだよ。 ベティ: あー、そういうことか！（微笑んで）ありがとうございます、先生。何だか少しスッキリしました。 アキラ: こちらこそ、話ができて楽しかったよ。さあ、次はどんな問題を解決しようかね？（二人は笑顔で握手をする。） --- このシーンは、数学の基礎である整数と浮動小数点数の違いに焦点を当てつつ、ユーモアを交えながら観客に分かりやすく伝えることを目指しています。また、対話を通じて歴史的な背景や小数の実用性について学ぶ機会を提供します。約5分間という制約内で、物語の一貫性と楽しさを保つことができました。 user: 第二次世界大戦後の日本の演劇界における社会的・政治的変化が、現代の作品にどのような形で反映されていますか？ assistant: 第二次世界大戦後、日本は大きな社会変革と経済成長を経験し、その影響は演劇作品にも深く反映されました。まず、戦争の悲惨さや平和の重要性がテーマとして多く取り上げられるようになりました。『夕鶴』や『桜の園』といった古典的な作品も再解釈され、新しい視点から上演されることが増えました。これにより、観客はより深い理解と共感を覚えることができました。 また、戦後の民主主義の進展に伴い、個人の権利や社会正義が強調されるようになったことも影響しています。例えば、井上ひさしの戯曲には、戦時中の抑圧からの解放や自由を求める人々の姿が描かれています。彼の作品はしばしばユーモアを交えつつも、鋭い社会批判を含んでおり、観客に対して考えさせる力を持っています。 さらに、女性の社会進出が進む中で、女性の視点を取り入れた作品が数多く生まれました。たとえば、岸田國士の『紙風船』では、家庭内の権力や性別役割についての議論が展開されます。このようなテーマは、現代のフェミニズム運動とも共鳴しており、時代を超えて共感を呼ぶものとなっています。 加えて、グローバリゼーションの影響も無視できません。海外の文化との交流が増えることで、異文化間の理解をテーマにした作品も登場しました。村上春樹の小説を原作とした舞台作品では、多国籍なキャラクターが登場し、異なる価値観や文化背景を持つ人々がどのように共存するかを探求する内容が多く見られます。 このように、戦後の日本の演劇作品は、歴史的背景や社会の変化を反映しつつ、多様なテーマを扱うようになっています。その結果、観客にとっても普遍的で今日的な問題を考えるきっかけとなるような豊かな表現の場が提供されているのです。"
txt="user: 次の課題は、あなたの学校の美術部で取り組む「美術史と技法の発展」をテーマにした展示会の企画です。この展示会は3ヶ月後に開催予定で、以下のような詳細なスケジュールを作成してください： 1. プロジェクトの目的を明確にし、どのようなテーマやコンセプトにするかを決定してください（例：「ルネサンスから現代アートまでの技法の進化」）。 2. 各セクションの展示内容を決め、どの時代・技法をカバーするかを計画してください。例えば、「油絵」、「水彩画」、「彫刻」、「デジタルアート」など。 3. アーティストの選定を行い、それぞれのアーティストがどの作品を展示するのかを決めてください。また、アーティストの背景情報やインタビューも準備します。 4. 会場レイアウトを設計し、各作品の配置場所を決めます。来場者が見やすいように工夫しましょう。 5. 宣伝活動の計画を立てます。ポスターのデザイン、SNSでの発信方法、学校内外での告知イベントなどを計画してください。 6. 予算の見積もりを作成し、必要な資金を確保する方法を考えてください。スポンサーシップやアーティストランスポンサーの募集なども検討してください。 7. 展示会当日の運営計画を立てて、スタッフの役割分担を行います。受付対応、案内役、解説員などの配置を考えましょう。 8. 最後に、展示会終了後のフィードバックを収集し、次回以降の改善点を洗い出してください。 この課題を通じて、チームワークの重要性やプロジェクト管理のスキルを身につけてください。期限内に完成させることを目指し、各自の担当分野について責任を持って取り組んでください。 assistant: それでは、具体的な計画を以下にまとめましたのでご確認ください。 ### 「美術史と技法の発展」展の企画書 #### 目的とテーマ設定 本展覧会の目的は、美術の歴史的な発展とその技術的進化を鑑賞者に理解してもらうことです。「ルネサンスから現代アートまでの技法の進化」をテーマに掲げ、多様な時代背景とアーティストたちの創造力を紹介します。 #### 展示内容の決定 - ルネサンス: - レオナルド・ダ・ヴィンチの油絵作品（「モナ・リザ」） - ラファエロの水彩画作品（「アテナイの学堂」） - バロック: - レンブラントの彫刻作品（「夜警」） - ルーベンスの絵画作品（「キリスト降架」） - ロココ: - フランソワ・ブーシェのデジタルアート作品（「ディアナとニンフ」） - 新古典主義: - ジャック＝ルイ・ダヴィッドの肖像画作品（「ナポレオンの戴冠式」） - 印象派: - エドゥアール・マネの風景画作品（「草上の昼食」） - キュビズム: - パブロ・ピカソの抽象画作品（「ゲルニカ」） - ポップアート: - アンディ・ウォーホルのキャンバスプリント作品（「マリリン・モンロー」） - 現代アート: - 村上隆のデジタルインスタレーション作品（「スーパーフラット」） #### アーティストの選定と展示品 - レオナルド・ダ・ヴィンチ: 「モナ・リザ」（油絵） - ラファエロ: 「アテナイの学堂」（水彩画） - レンブラント: 「夜警」（彫刻） - ルーベンス: 「キリスト降架」（絵画） - フランソワ・ブーシェ: 「ディアナとニンフ」（デジタルアート） - ジャック＝ルイ・ダヴィッド: 「ナポレオンの戴冠式」（肖像画） - エドゥアール・マネ: 「草上の昼食」（風景画） - パブロ・ピカソ: 「ゲルニカ」（抽象画） - アンディ・ウォーホル: 「マリリン・モンロー」（キャンバスプリント） - 村上隆: 「スーパーフラット」（デジタルインストール） #### 会場レイアウト設計 会場は学校の体育館を使用し、以下のゾーンに分けます。 - 入口エリア: チケット販売およびパンフレット配布所を設置。 - ルネサンスゾーン: 壁面に大きなパネルを設置し、歴史的な背景を説明するテキストを配置。 - バロックゾーン: 中規模なスペースを設け、立体作品を展示。 - ロココゾーン: 明るく開放感のある空間にデジタルアートを展示。 - 新古典主義ゾーン: 落ち着いた雰囲気の中で肖像画を中心に展示。 - 印象派ゾーン: 自然光を取り入れた明るい部屋で風景画を展示。 - キュビズムゾーン: モノクロームの色調で抽象画を展示。 - ポップアートゾーン: 大胆なデザインの壁面を使い、キャンバスプリントを展示。 - 現代アートゾーン: インタラクティブな要素を含むデジタルインスタレーションを展示。 #### 宣伝活動の計画 - SNSキャンペーン: Instagram, Twitter, Facebookを活用し、定期的に更新。ハッシュタグを設定。#ArtEvolution2023を使用。 - ポスター制作: 高解像度のビジュアルを用いたA2サイズのポスターを制作し、校内外の掲示板に掲載。 - 告知イベント: 地元のアートギャラリーと提携し、共同イベントを主催。 #### 予算見積もりと資金調達 総予算は約¥500,000と見積もります。主な収入源としてスポンサーシップを募集し、企業広告を掲載したパンフレットを作成。さらに、クラウドファンディングサイトを利用して個人支援を募る予定です。 #### 展示会当日の準備 スタッフは次のように役割を分担します。 - 受付担当: 入場者の管理とチケットの確認を行う。 - ガイドツアー担当者: 見学者のグループを案内し、各作品の説明をする。 - 解説員: 特定のテーマに関する深い知識を提供し、質問に対応する。 - 清掃係: 常に清潔な状態を保つために巡回する。 #### フィードバック収集と改善策 展示会終了後、アンケートを実施し、来訪者からの意見を集めます。特に良かった点と改善すべき点を具体的に洗い出し、次回以降の展覧会に生かしていきます。 以上が「美術史と技法の発展」展の詳細な企画書となります。成功に向けて全力を尽くしますので、よろしくお願いいたします。 user: 展示会で取り上げるアーティストを選定する際に考慮した基準は何ですか？ assistant: 展示会においてどのアーティストを取り上げるかを決定する際には、いくつかの重要な基準を考慮しました。まず第一に、そのアーティストの作品が現代社会や文化にどのような影響を与えているかを評価します。これは作品が持つメッセージ性や社会的意義を重視するものです。次に、技術的な卓越性も大きな要素です。作品の質や独自性、創造的なアプローチなどが優れているかどうかを確認します。また、過去の業績や受賞歴も参考にしますが、それだけでなく新しい挑戦を続けているかどうかにも注目しています。さらに、観客層との親和性を考え、幅広い層に受け入れられる可能性があるかも検討材料とします。最後に、地域や社会の多様性を反映させるために、異なる背景を持つアーティストを積極的に選ぶことも意識しています。これらの基準を組み合わせて総合的に判断し、バランスの取れたラインナップを構築することを目指しました。"
chunk=tokenizer.tokenize(txt)
len(chunk)

1799

In [ ]:

run_dpo(model,tokenizer,dpo_dataset,
        run_name="test")

In [9]:
model.save_pretrained(f"out/{generation}")